In [3]:
!pip3 install Cinemagoer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 6.2 MB/s eta 0:00:00


In [5]:
import tweepy
import json
import csv
from imdb import Cinemagoer
import time

# Storing all the credential of twitter developer account
api_key ='7IdbAMeBxMXCU0V7h2xkaMXGP'
api_key_secret = '95QRiLs7omec37JMpnyNyOgHg7c2izlhNUnc0KRY5OlTK7yt6c'
access_token = '551048477-MPRU0VSVxugYrVgfIIqw380rRZuEemGyJi9q7SNs'
access_token_secret = 'leLo4J0LhubtqdW1ZC4nJj6cbHm2eB8KcC6aPm8ScFE4j'

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

def search_movie_tweets(movie_name, tweet_count = 50):
    query = f"{movie_name} movie -filter:retweets"
    tweets = tweepy.Cursor(api.search_tweets, q=query, lang="en", tweet_mode="extended").items(tweet_count)
    return [tweet.full_text for tweet in tweets]


ia = Cinemagoer()

def get_movie_info(movie_name):
    movies = ia.search_movie(movie_name)
    if not movies:
        return None

    movie = ia.get_movie(movies[0].movieID)
    
    return {
        "title": movie.get("title"),
        "genres": movie.get("genres"),
        "rating": movie.get("rating"),
    }

def collect_movie_data(movie_name):
    movie_info = get_movie_info(movie_name)
    if not movie_info:
        return None

    tweets = search_movie_tweets(movie_name)

    return {
        "movie_name": movie_info["title"],
        "rating": movie_info["rating"],
        "genres": movie_info["genres"],
        "tweets": tweets
    }

def save_movie_data_to_csv(movie_data_list, filename):
    with open(filename, "w", newline='', encoding='utf-8') as csvfile:
        fieldnames = ['movie_name', 'rating', 'genres', 'tweet']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for movie_data in movie_data_list:
            movie_name = movie_data['movie_name']
            rating = movie_data['rating']
            genres = ",".join(movie_data['genres'])
            for tweet in movie_data['tweets']:
                writer.writerow({
                    'movie_name': movie_name,
                    'rating': rating,
                    'genres': genres,
                    'tweet': tweet
                })

# Read movie names from the JSON file
with open("movies.json", "r") as file:
    movies = json.load(file)


batch_size = 50
total_movies = len(movies)
current_movie_index = 0

movie_data_list = []

while current_movie_index < total_movies:
    # get the next batch of movies to process
    movie_batch = movies[current_movie_index:current_movie_index+batch_size]
    
    # process each movie in the batch
    for movie in movie_batch:
        movie_data = collect_movie_data(movie)
        if movie_data:
            movie_data_list.append(movie_data)
        
    # increment the current movie index by the batch size
    current_movie_index += batch_size
    
    # wait for 15 minutes before processing the next batch
    time.sleep(900)

# Save the movie data to a CSV file
save_movie_data_to_csv(movie_data_list, "movie_data.csv")


TooManyRequests: ignored

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def preprocess_tweet(tweet, movie_names):
    # Remove movie names
    for movie_name in movie_names:
        tweet = re.sub(movie_name, '', tweet, flags=re.IGNORECASE)

    # Remove URLs, mentions, hashtags, and emojis
    tweet = re.sub(r'http\S+|www\S+|https\S+|\@\w+|\#|[\U00010000-\U0010ffff]', '', tweet, flags=re.MULTILINE)

    # Convert to lowercase
    tweet = tweet.lower()

    # Tokenize
    tokens = word_tokenize(tweet)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]

    return tokens


tweets = pd.read_csv('movie_data.csv')
movie_names = tweets['movie_name'].tolist()
tweets['preprocessed_tweets'] = tweets['tweet'].apply(preprocess_tweet, movie_names=movie_names)
tweets.to_csv('preprocessed_tweets.csv', index=False)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nagasriramkochetti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nagasriramkochetti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nagasriramkochetti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from textblob import TextBlob


# load the preprocessed tweets from a CSV file
tweets_df = pd.read_csv('preprocessed_tweets.csv')
count_rows = len(tweets_df["preprocessed_tweets"])

# create a list to store the sentiment scores
sentiment_scores = []
sentiment_text = []
averages_by_item = []
# iterate over each tweet
for tweet in tweets_df['preprocessed_tweets']:
    # create a TextBlob object for the tweet
    blob = TextBlob(tweet)
    
    # calculate the sentiment polarity score
    sentiment_scores.append(blob.sentiment.polarity)

    
# add the sentiment scores to the tweets dataframe
tweets_df['sentiment_score'] = sentiment_scores

df = pd.DataFrame(tweets_df, columns=['movie_name','genres','sentiment_score'])

averages_by_item = df.groupby(['movie_name','genres'])['sentiment_score'].mean().reset_index()

distinct_movie = pd.DataFrame({'movie_name': df['movie_name'].unique()})

distinct_genre = pd.DataFrame({'genres': df['genres'].unique()})
score = pd.merge(distinct_movie, distinct_genre, how='cross')
score = pd.merge(score, averages_by_item, on=['movie_name', 'genres'], how='left')

result = pd.DataFrame(score)
result = result.dropna()

for score in result['sentiment_score']:
    if(score > 0.1 ):
      {
          sentiment_text.append("Good")
      }
    elif(score < -0.1):
      {
          sentiment_text.append("Bad")
      }
    else:
      {
          sentiment_text.append("Average")
      }  
     
result['average_sentiment'] = sentiment_text
df100 = pd.DataFrame(result, columns=['movie_name','genres','average_sentiment','sentiment_score'])

# print the top 10 tweets with the highest sentiment score

df100.to_csv('sentiment.csv', index=False)

In [9]:
import pandas as pd
from sklearn.cluster import KMeans


movies_df = pd.read_csv('sentiment.csv')
movies_df = pd.DataFrame(movies_df)

# One-hot encode the 'genres' and 'average_sentiment' columns
genres_one_hot = movies_df['genres'].str.get_dummies(',')
sentiment_one_hot = pd.get_dummies(movies_df['average_sentiment'])

# Combine the one-hot encoded features into a single dataframe
features = pd.concat([genres_one_hot, sentiment_one_hot, movies_df['sentiment_score']], axis=1)

# Perform clustering using KMeans with 3 clusters
kmeans = KMeans(n_clusters=3).fit(features)

# Add the cluster labels to the original dataframe
movies_df['cluster_label'] = kmeans.labels_

# Write the resulting clusters to separate sheets in the same Excel file
with pd.ExcelWriter('sentiment.xlsx', mode='w') as writer:
    # Write the original sheet with cluster label column
    movies_df.to_excel(writer, sheet_name='Movies', index=False)
    # Write the resulting clusters to separate sheets
    for label in movies_df['cluster_label'].unique():
        cluster_df = movies_df[movies_df['cluster_label'] == label][['movie_name', 'genres', 'average_sentiment', 'sentiment_score']]
        cluster_df['cluster_label'] = label
        cluster_df.to_excel(writer, sheet_name=f'Cluster {label}', index=False)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, render_template, request
from IPython.display import display, HTML


# Load movie data
movies_df = pd.read_csv('sentiment.csv')

# Combine genres and average_sentiment as a single string
movies_df['combined_features'] = movies_df['genres'] + ' ' + movies_df['average_sentiment']

# Create a CountVectorizer and fit_transform the combined_features
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(movies_df['combined_features'])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix)

def get_title_from_index(index):
    return movies_df[movies_df.index == index]['movie_name'].values[0]

def get_index_from_title(title):
    return movies_df[movies_df.movie_name == title].index.values[0]

def recommend_movies(movie_title):
    movie_index = get_index_from_title(movie_title)
    similar_movies = list(enumerate(cosine_sim[movie_index]))
    sorted_similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    
    recommended_movies = []
    for movie in sorted_similar_movies[1:6]:
        recommended_movies.append(get_title_from_index(movie[0]))
    return recommended_movies
